# Outline of Finance Problem and Numerical Procedure


**Asian** and **Lookback options**,are path-dependent, meaning their payoff depends on the entire history of the underlying asset's price during the option’s life.

1. **Asian Options**: For Asian options, the payoff is determined by the average price of the underlying asset over a specific time period. This averaging feature reduces the impact of price volatility compared to traditional European options. The payoff for an Asian call option is:
   $$
   \text{Payoff} = \max\left(\frac{1}{n} \sum_{i=1}^n S_i - E, 0\right)
   $$
   where S_i are the asset prices at different times during the option’s life, and E is the strike price.

2. **Lookback Options**: Lookback options allow the holder to "look back" at the underlying asset's price path and choose the optimal price for determining the payoff. For a floating strike lookback call, the payoff is:
   $$
   \text{Payoff} = \max(S_T - \min(S_i), 0)
   $$
   where S_T is the price at expiry, and S_i is the minimum price observed over the life of the option.

Both option types are priced by calculating the **expected value** of the discounted payoff under the risk-neutral measure:
$$
V(S_0, t) = e^{-r(T-t)} \mathbb{E}_Q [\text{Payoff}(S_T)]
$$
where r is the risk-free rate, T - t is the time to expiry, and E_Q represents the expectation under the risk-neutral measure.

#### NumericalProcedure:
To solve this problem, I implemented a **Monte Carlo simulation** coupled with the **Euler-Maruyama method** to simulate the underlying asset price. The procedure follows these steps:

1. **Simulating Stock Prices**:
   The stock prices are simulated using the **Geometric Brownian Motion (GBM)** model:
   $$
   dS_t = r S_t dt + \sigma S_t dW_t
   $$
   where r is the risk-free rate, sigma is the volatility, and W_t is a Wiener process (Brownian motion).

   Using the **Euler-Maruyama discretization**, the stock price at each time step is approximated as:
   $$
   S_{t + \Delta t} = S_t \left(1 + r \Delta t + \sigma \sqrt{\Delta t} Z\right)
   $$
   where Z is a standard normal random variable, and Delta t is the time increment.

2. **Monte Carlo Simulation**:
   I generated numerous simulated paths for the stock price, and for each path, I computed the option payoff depending on the type of option (Asian or Lookback). The expected payoff is then obtained by averaging the payoffs from all simulations.

3. **Discounting the Payoff**:
   The final option price is calculated by discounting the average payoff back to the present using the risk-free rate:
   $$
   \text{Option Price} = e^{-r(T-t)} \cdot \frac{1}{N} \sum_{i=1}^N \text{Payoff}_i
   $$
   where \(N\) is the number of simulated paths.

#### Parameters Used:
For the base case, I used the following parameters:
- Initial stock price:$$ S_0 = 100$$
- Strike price: $$E = 100$$
- Time to expiry: $$ T - t = 1 \text{ year} $$
- Volatility:$$ sigma = 20 $$
- Risk-free interest rate:$$ r = 5\% $$


#### Numerical Considerations:
​
To ensure accuracy, a large number of simulation paths (\( N \)) are required. The choice of time step \( \Delta t \) and the number of simulated paths significantly affect the precision of the results. Additionally, the variance of the results can be high for exotic options like lookback options, so variance reduction techniques such as antithetic variates or control variates could be applied for more stable results.
​
#### Summary
​
In this project, the Monte carlo method using the E-M scheme was employed to price exotic options. This approach provides flexibility in modeling path-dependent options such as Asian and lookback options. However, it also comes with computational challenges due to the large number of paths required for accurate results.


# Simulation

In [1]:
import numpy as np
import pandas as pd

# Parameters
E_classic = 100  
T = 1           
r = 0.05        
N = 252         
M = 1000000        
dt = T / N      
sigma = 0.2     


def simulate_stock_price(S0, r, sigma, T, N, M):
    dt = T / N
    Z = np.random.standard_normal((M, N))  # M paths, N time steps
    S = np.zeros((M, N+1))
    S[:, 0] = S0
    increments = (r * dt) + (sigma * np.sqrt(dt) * Z)
    S[:, 1:] = S0 * np.exp(np.cumsum(increments, axis=1))
    return S


def asian_call_arithmetic_payoff(S, E):
    avg_S = np.mean(S, axis=1)  
    payoff = np.maximum(avg_S - E, 0)
    return payoff

def market_making_crypto_option(S, E=None):
    
    avg_S_first_10 = np.mean(S[:, :11], axis=1)  
    
   
    strike_price = avg_S_first_10
    
    
    payoff = np.maximum(S[:, -1] - strike_price, 0)
    
    return payoff


def asian_call_geometric_payoff(S, E):
    geom_S = np.exp(np.mean(np.log(S), axis=1))  
    payoff = np.maximum(geom_S - E, 0)
    return payoff


def lookback_call_payoff(S, E):
    max_S = np.max(S, axis=1)
    payoff = np.maximum(max_S - E, 0)
    return payoff


def calculate_option_prices(S0, sigma, T):
    
    S = simulate_stock_price(S0, r, sigma, T, N, M)
    prices = []
    for option_type, payoff_func in [
        ("Asian Arithmetic", asian_call_arithmetic_payoff),
        ("Asian Geometric", asian_call_geometric_payoff),
        ("Lookback", lookback_call_payoff),
        ("Crypto MM", market_making_crypto_option)
    ]:
        price = np.exp(-r * T) * np.mean(payoff_func(S[:, 1:], E_classic))
        prices.append({
            "Stock Price (S0)": S0,
            "Volatility": sigma,
            "Option Type": option_type,
            "Option Price": price,
            "Time to Maturity":T
        })

    return prices



# Results

In [2]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go


S0_values = [70,80, 90, 100, 110, 120, 130]  


volatility_values = [0.0, 0.05, 0.1, 0.15, 0.20, 0.25, 0.30, 0.35, 0.4] 

#Time values to vary over
T_values = [0,0.125, 0.25,0.375, 0.5, 0.675, 0.75, 0.875, 1] 


option_prices_arithmetic = np.zeros((len(T_values), len(S0_values), len(volatility_values)))
option_prices_geometric = np.zeros((len(T_values), len(S0_values), len(volatility_values)))
option_prices_lookback = np.zeros((len(T_values), len(S0_values), len(volatility_values)))
option_prices_crypto = np.zeros((len(T_values), len(S0_values), len(volatility_values)))


for t_idx, T in enumerate(T_values):
    for i, S0 in enumerate(S0_values):
        for j, sigma in enumerate(volatility_values):
            result_set = calculate_option_prices(S0, sigma, T)
            for result in result_set:
                if result['Option Type'] == "Asian Arithmetic":
                    option_prices_arithmetic[t_idx, i, j] = result['Option Price']
                elif result['Option Type'] == "Asian Geometric":
                    option_prices_geometric[t_idx, i, j] = result['Option Price']
                elif result['Option Type'] == "Lookback":
                    option_prices_lookback[t_idx, i, j] = result['Option Price']
                elif result['Option Type'] == "Crypto MM":
                    option_prices_crypto[t_idx, i, j] = result['Option Price']


S0_grid, volatility_grid = np.meshgrid(S0_values, volatility_values)

# plot surfaces
fig = go.Figure()

for t_idx, T in enumerate(T_values):
    fig.add_trace(go.Surface(
        z=option_prices_arithmetic[t_idx].T,
        x=S0_grid,
        y=volatility_grid,
        colorscale='Viridis',
        name=f'Asian Arithmetic, T={T} years',
        hovertemplate='Option: Asian Arithmetic <br>S0: %{x}<br>Volatility: %{y}<br>Option Price: %{z}<extra></extra>',  # Add option name
        visible=(t_idx == 0)
    ))
    fig.add_trace(go.Surface(
        z=option_prices_geometric[t_idx].T,
        x=S0_grid,
        y=volatility_grid,
        colorscale='Cividis',
        name=f'Asian Geometric, T={T} years',
        hovertemplate='Option: Asian Geometric <br>S0: %{x}<br>Volatility: %{y}<br>Option Price: %{z}<extra></extra>',  # Add option name
        visible=(t_idx == 0)
    ))
    fig.add_trace(go.Surface(
        z=option_prices_lookback[t_idx].T,
        x=S0_grid,
        y=volatility_grid,
        colorscale='Plasma',
        name=f'Lookback, T={T} years',
        hovertemplate='Option: Lookback <br>S0: %{x}<br>Volatility: %{y}<br>Option Price: %{z}<extra></extra>',  # Add option name
        visible=(t_idx == 0)
    ))
    fig.add_trace(go.Surface(
        z=option_prices_crypto[t_idx].T,
        x=S0_grid,
        y=volatility_grid,
        colorscale='Inferno',
        name=f'Crypto MM, T={T} years',
        hovertemplate='Option: Crypto_MM <br>S0: %{x}<br>Volatility: %{y}<br>Option Price: %{z}<extra></extra>',  # Add option name
        visible=(t_idx == 0)
    ))


steps = []
for t_idx, T in enumerate(T_values):
    step = dict(
        method="update",
        args=[{"visible": [False] * len(fig.data)}],  
        label=f'T={T:.1f} years'
    )
    step["args"][0]["visible"][t_idx * 4] = True  
    step["args"][0]["visible"][t_idx * 4 + 1] = True  
    step["args"][0]["visible"][t_idx * 4 + 2] = True
    step["args"][0]["visible"][t_idx * 4 + 3] = True 
    steps.append(step)


sliders = [dict(
    active=0,
    currentvalue={"prefix": "Time to Maturity (T): "},
    pad={"t": 50},
    steps=steps
)]

fig.update_layout(
    sliders=sliders,
    title="Option Prices with Varying S0, Volatility, and Time to Maturity (T)",
    scene=dict(
        xaxis_title="Stock Price (S0)",
        yaxis_title="Volatility",
        zaxis_title="Option Price"
    ),
    autosize=True,
    width=900,
    height=900,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()

In [3]:
table_data_arithmetic = []
table_data_geometric = []
table_data_lookback = []
table_data_crypto = []


for t_idx, T in enumerate(T_values):
    for i, S0 in enumerate(S0_values):
        for j, sigma in enumerate(volatility_values):
            table_data_arithmetic.append([T, S0, sigma, option_prices_arithmetic[t_idx, i, j]])
            table_data_geometric.append([T, S0, sigma, option_prices_geometric[t_idx, i, j]])
            table_data_lookback.append([T, S0, sigma, option_prices_lookback[t_idx, i, j]])
            table_data_crypto.append([T, S0, sigma, option_prices_crypto[t_idx, i, j]])
            


df_arithmetic = pd.DataFrame(table_data_arithmetic, columns=["Time to Maturity (T)", "Stock Price (S0)", "Volatility (σ)", "Asian Arithmetic Option Price"])
df_geometric = pd.DataFrame(table_data_geometric, columns=["Time to Maturity (T)", "Stock Price (S0)", "Volatility (σ)", "Asian Geometric Option Price"])
df_lookback = pd.DataFrame(table_data_lookback, columns=["Time to Maturity (T)", "Stock Price (S0)", "Volatility (σ)", "Lookback Option Price"])
df_crypto = pd.DataFrame(table_data_crypto, columns=["Time to Maturity (T)", "Stock Price (S0)", "Volatility (σ)", "Crypto Option Price"])

df= df_arithmetic
df["Asian Geometric Option Price"]= df_geometric["Asian Geometric Option Price"]
df["Lookback Option Price"] = df_lookback["Lookback Option Price"]
df['Crypto Option Price']= df_crypto['Crypto Option Price']

vary_s0 = df[(df['Time to Maturity (T)']==1.0)& (df['Volatility (σ)']==0.20)]
vary_t = df[(df['Stock Price (S0)']==100)& (df['Volatility (σ)']==0.20)]
vary_sigma = df[(df['Stock Price (S0)']==100)& (df['Time to Maturity (T)']==1.0)]

In [4]:
vary_s0

,Time to Maturity (T),Stock Price (S0),Volatility (σ),Asian Arithmetic Option Price,Asian Geometric Option Price,Lookback Option Price,Crypto Option Price
508,1.0,70,0.2,0.011597,0.007199,0.947330,8.097267
517,1.0,80,0.2,0.250984,0.205796,3.795516,9.240901
526,1.0,90,0.2,1.840363,1.696380,9.984175,10.427647
535,1.0,100,0.2,6.401790,6.160499,19.652558,11.528970
544,1.0,110,0.2,14.047248,13.721574,31.163216,12.746011
553,1.0,120,0.2,23.279981,22.883926,42.640165,13.907485
562,1.0,130,0.2,32.996671,32.550132,54.088424,15.025813


In [5]:
vary_t

,Time to Maturity (T),Stock Price (S0),Volatility (σ),Asian Arithmetic Option Price,Asian Geometric Option Price,Lookback Option Price,Crypto Option Price
31,0.000,100,0.2,0.000000,4.263256e-14,0.000000,0.000000
94,0.125,100,0.2,1.856256,1.832260e+00,5.941904,3.221973
157,0.250,100,0.2,2.760872,2.710086e+00,8.736527,4.829138
220,0.375,100,0.2,3.499348,3.420097e+00,10.991218,6.162022
283,0.500,100,0.2,4.159264,4.050151e+00,12.991380,7.366656
346,0.675,100,0.2,5.005368,4.851469e+00,15.527422,8.937652
409,0.750,100,0.2,5.330603,5.157724e+00,16.501819,9.543157
472,0.875,100,0.2,5.882343,5.675448e+00,18.128827,10.568540
535,1.000,100,0.2,6.401790,6.160499e+00,19.652558,11.528970


In [6]:
vary_sigma

,Time to Maturity (T),Stock Price (S0),Volatility (σ),Asian Arithmetic Option Price,Asian Geometric Option Price,Lookback Option Price,Crypto Option Price
531,1.0,100,0.00,2.427886,2.417725,4.877058,4.763730
532,1.0,100,0.05,2.779942,2.751748,6.928292,5.278789
533,1.0,100,0.10,3.826572,3.753731,10.750825,7.031550
534,1.0,100,0.15,5.070247,4.926826,15.052988,9.200113
535,1.0,100,0.20,6.401790,6.160499,19.652558,11.528970
536,1.0,100,0.25,7.869719,7.496746,24.694714,14.187195
537,1.0,100,0.30,9.393883,8.857552,30.017536,16.951885
538,1.0,100,0.35,11.031959,10.291364,35.797899,19.985752
539,1.0,100,0.40,12.726849,11.744317,41.872191,23.161678


# Observations 

In the code above I have simulated the potential prices of common types of Asian and Lookback options. As expected with call options several behaviors have been carried over from the vanilla counterparts:
* With **varying time to expiry** we see that the further we are from expiry (in this case t=1) all options are worth more than their counterparts with shorter expiries
* With **varying initial price**  the outcome is also predictable with in the money options being worth more than the out of the money ones and also displaying a predictable path in terms of value
* With **varying volatility** once moer the results provided a behavuiour that we would have expected of a vanilla call option, with the value increasing as sigma also increased. 

What was truly interesting is the difference in value of these options due to the path dependent nature of their payoffs:

* **Lookback vs Asian:** These two types of options present a stark differnce in how they respond to variations in initial pricce and volatility with the lookback options being much more sensitive in variations of these metics where at the extremes of this analysis the price of the lookback option could be several multiples higher than the Asian option. 
* **Geometric vs Arithmetic** Due to how the average is computed, there are also slight differences emerging from these two asian options. In this simulation, the main driver of this divergence is the starting price. I expect these options to converge for a given price if we increased the time to expiry as it would reduce the weight that a singe starting point could have on the overall average of an option over its lifetime

Within this notebook I have included a different type of option called **"Crypto Option"**. Within my industry, cryptocurrency marketmakers use a type  of option where the strike of the option is set after several days (in this case 10) since the start of the marketmaking mandate. This structure aligns the client's interest and the funds interest to provide liquidity throughout the entire duration of the marketmaking contract as well as provide upside exporsure should the token be succesful. When paired with a fiat denominated loan in the form of the stablecoin pair that the fund will be quoting the token against, it provides the fund downside protection as it will be using the borrowed money to buy the token should the project not be succesful, while leaving the funds balance sheet relatively protected. I have decided to include this in this assignment as it takes parts of both asian and lookback options but applies them to another element of the option structure.

* We can see form this option that the behaviour is very close to the one of a simple call option, however it provides further upside potential as the strike price is variable and as such could lead to even greater returns for a given price at S_T should the initial average price be lower than the 100 strike price.given in the base scenario.



# Limitations of the simulation

Monte Carlo simulation is a versatile method for pricing complex, path-dependent options like **Asian** and **Lookback options**. However, there are several key limitations specific to these options when using Monte Carlo, and each has corresponding strategies to mitigate the challenges:

#### 1. Computational Complexity and Time
Monte Carlo methods require a large number of simulations for convergence, especially for path-dependent options like Asian and Lookback options. Since each simulation requires simulating the entire price path, this makes the method computationally intensive. For these options, particularly lookback options (which depend on the minimum or maximum price throughout the option’s life), a higher number of time steps and simulations are needed to reduce bias.

**Mitigation**: To mitigate thislimitation, we can use **variance reduction techniques** such as antithetic variates and control variates to reduce the number of simulations required for accurate results. Additionally, **parallel processing** can be employed to run multiple simulations concurrently, significantly reducing computation time.

#### 2. Numerical Accuracy and Discretization Errors
The Euler-Maruyama method used for simulating stock price paths introduces discretization errors because it approximates continuous price movements with discrete time steps. This can be problematic for both Asian and Lookback options:
- **Asian options**: The average price is calculated based on discrete time steps, which can lead to inaccuracies if the time grid is not fine enough.
- **Lookback options**: Since these depend on the minimum or maximum price over the life of the option, discretization leads to an approximation of the true minimum or maximum, which could significantly affect the option price.

**Mitigation**: Increasing the number of time steps can help to capture more accurate price paths, reducing the likelihood of missing key minor max.

#### 3. Model assumptions
In this context, Monte Carlo simulations assume that the underlying asset follows a **geometric Brownian motion (GBM)** with constant volatility and a constant risk-free rate. However, real-world markets often exhibit features such as **stochastic volatility**, **jumps**, and **changing interest rates**. This can lead to inaccuracies in pricing, particularly for options like Lookback options, which are sensitive to extreme price movements.

**Mitigation**: A more sophisticated model, such as the **Heston stochastic volatility model** or **Merton's jump diffusion model**, could better reflect realworld dynamics for these options. These models are more complex and computationally intensive but porvide a more accurate pricing framework for options like lookback options, where sudden price jumps or volatility changes have a significant impact.

#### 4. Lack of Analytical Solutions Affects Greeks computations
Monte Carlo methods do not provide closed-form solutions, which are available for vanilla options. For Asian and Lookback options, there are no general analytical solutions, this makes it difficult to quickly obtain prices and to compute the greeks

**Mitigation**: While Monte Carlo lacks closed-form solutions, the **pathwise method** or **likelihood ratio method** can be used to compute the Greeks directly from the simulated paths. This allows for the estimation of option sensitivities like delta, gamma, and vega, though with additional computational effort.

#### 5. Capturing Rare Events
Lookback options, in particular, may be sensitive to rare market events, such as extreme price drops or rallies. Capturing such events accurately in a Monte Carlo simulation requires simulating a large number of paths, which can be computationally expensive.

**Mitigation**: **Importance sampling** is a technique that can help capture rare events more efficiently by increasing the likelihood that important but rare scenarios are sampled in the simulation. This reduces the need to run an enormous number of simulations to capture extreme price movements, improving the accuracy of Lookback option pricing without excessive computational overhead


# Conclusions

This notebook explored the pricing of various exotic options—Asian Arithmetic, Asian Geometric, Lookback, and a Crypto Market-Making (Crypto MM) option—using Monte Carlo simulations. Several key observations and limitations were noted throughout the analysis:

#### 1. Performance of Option Pricing Models

- **Asian Arithmetic and Geometric Options**: The pricing of both Asian Arithmetic and Geometric options reveals that their prices tend to increase with stock price ($S_0$) and volatility. This behavior is consistent with their payoff structure, where the average price (arithmetic or geometric) reduces the impact of extreme stock price fluctuations, leading to relatively lower option prices than European options. The Geometric Asian option consistently priced lower than the Arithmetic due to the geometric average, which is always less than or equal to the arithmetic average for the same dataset.
  
- **Lookback Option**: The Lookback option, which provides the holder the right to exercise based on the minimum or maximum stock price observed, demonstrated higher sensitivity to volatility. As expected, the ability to "look back" and pick an optimal price makes the option more valuable, particularly in high-volatility environments where the stock price deviates significantly. However, this flexibility also introduces computational challenges in pricing due to the non-linearity in the payoff function, as evidenced by the higher variance in results compared to the Asian options.

#### 2. Monte Carlo Simulation Observations

- The Monte Carlo simulation effectively captured the option prices across a range of volatility values and stock prices. However, it required a significant number of paths to stabilize the results
  
- **Time to Maturity**: As expected, option prices generally increased with time to maturity, since a longer time frame introduces more uncertainty and thus more opportunity for the option to end in-the-money.


#### 3. Final Recommendations

Given the observations, the following recommendations are made for future improvements:

- **Advanced Volatility Models**: Incorporating stochastic volatility models could improve pricing accuracy for options like the lookback and Crypto where constant volatility assumptions are unrealistic.
  
- **Parallel Computing**: The simulation performance can be enhanced by leveraging parallel computing methods to distribute the computational load across multiple processors, reducing simulation time while maintaining accuracy.

In conclusion, the Monte Carlo method provided a solid framework for pricing these exotic option though its limitations highlight the need for more advanced models and techniques, especially when dealing with highly volatile assets like cryptocurrencies.


# References

* All the information used here is derived from the Module 3 Lectures and Python tutorials of the CQF Course - cqf.com 
* The references on the crypoto currency option are from my own experience working in the industry. 